In [1]:
import numpy as np
import open3d as o3d
import laspy
import glob
from pyproj import Transformer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Visualize point cloud from lidar data
## Load Data

In [2]:
# convert a las object into a point cloud x,y,z
def las_to_point(las):
    return np.stack([las.x, las.y, las.z], axis=0).transpose((1, 0))

# take 1 in "by" elements of array
def downsample(array, by:int): 
    indices = list(range(0,len(array),by))
    downsampled = np.take(array, indices, axis=0)
    return downsampled

# get point cloud from files
def load_data(datadir, downsampleby=1000):
    point_data = np.array([]).reshape(0,3)
    for file in glob.glob(datadir+"*.laz"):
        print("Loading", file)
        las = laspy.read(file)
        new_points = downsample(las_to_point(las), downsampleby)
        point_data = np.concatenate((point_data, new_points))
    print("Number of points:", point_data.shape[0])
    # convert to open3d point cloud
    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(point_data)
    return pc

In [4]:
datadir = "./data/MNS/"
pc = load_data(datadir, 1000)

Loading ./data/MNS/LHD_FXX_0798_6383_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0795_6382_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0795_6383_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0795_6384_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0795_6385_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0795_6386_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0796_6382_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0796_6383_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0796_6384_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0796_6385_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0796_6386_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0797_6382_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0797_6383_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0797_6384_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FXX_0797_6385_PTS_C_LAMB93_IGN69.copc.laz
Loading ./data/MNS/LHD_FX

In [5]:
# display test
o3d.visualization.draw_geometries([pc])

In [7]:
# other way to downsample, once the pc object is defined
#pc.voxel_down_sample(0.05)

## Define view point

In [7]:
# project a point from latitude longitude to Lambert 93 coordinates
def project_to_lambert93(lat, lon, altitude):
    projector = Transformer.from_crs("EPSG:4326", "EPSG:2154")
    view_point = np.array(projector.transform(lat,lon)+(altitude,))
    return view_point

lat = 44.5451545715
lon = 4.2165341377
elevation=780
view_point = project_to_lambert93(lat, lon, elevation)

In [8]:
# Add a sphere at view point
sphere = o3d.geometry.TriangleMesh.create_sphere(radius=15)
sphere.translate(view_point)
o3d.visualization.draw_geometries([pc, sphere])

In [10]:
width = 1280
height = 720
# change intrinsic and extrinsinc matrices in this file
parameters = o3d.io.read_pinhole_camera_parameters("ScreenCamera_viewpoint.json")

In [13]:
# example to set the camera to the registered location
vis = o3d.visualization.Visualizer()
vis.create_window(width=width, height=height)
vis.add_geometry(pc)
#vis.set_full_screen(True)
vis.get_view_control().convert_from_pinhole_camera_parameters(parameters, True)
vis.run()
vis.destroy_window()

## Set camera actions

In [21]:
# camera rotation around local up axis
def rotate_view_right(vis):
    ctr = vis.get_view_control()
    ctr.camera_local_rotate(4.0, 0.0)
    return False

def rotate_view_left(vis):
    ctr = vis.get_view_control()
    ctr.camera_local_rotate(-4.0, 0.0)
    return False

# camera translation towards local front axis
def translate_view(vis):
    ctr = vis.get_view_control()
    ctr.camera_local_translate(10,0,0)
    return False

# set the local up axis to be colinear to global z axis
def correct_up(vis):
    ctr = vis.get_view_control()
    ctr.set_up(np.array([0,0,1]))
    return False

# set the camera position to view point
def set_view(vis):
    ctr = vis.get_view_control()
    ctr.convert_from_pinhole_camera_parameters(parameters, True)
    return False

# set the local front axis to look at viewpoint
def look_at_view_point(vis):
    ctr = vis.get_view_control()
    ctr.set_lookat(view_point)
    return False

# def print_extrinsic(vis):
#     ctr = vis.get_view_control()
#     params = ctr.convert_to_pinhole_camera_parameters()
#     print(params.extrinsic)
#     return False

key_to_callback = {}
key_to_callback[o3d.visualization.gui.KeyName.RIGHT] = rotate_view_right
key_to_callback[o3d.visualization.gui.KeyName.LEFT] = rotate_view_left
key_to_callback[ord("T")] = translate_view
key_to_callback[ord("U")] = correct_up
key_to_callback[ord("V")] = set_view
key_to_callback[ord("L")] = look_at_view_point


Register camera position (to do once per viewpoint):
- place camera vertically to have viewpoint around the center (eventually use U)
- use L to lock the viewpoint direction
- use T to translate until viewpoint is reached
- use P to save the camera parameters

Place camera to viewpoint:
- use V to set view
- use left/right arrows to rotate until desired view is obtained

In [23]:
#o3d.visualization.draw_geometries_with_animation_callback([pc],rotate_view)
o3d.visualization.draw_geometries_with_key_callbacks([pc, sphere],key_to_callback, width=width, height=height)

In [126]:
# compute rotation matrix from vector
# import cv2
# rvec = np.array([0.0, 0.0, 0.0])
# R, _ = cv2.Rodrigues(rvec)
# R